In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
base_url="https://www.coffeereview.com"
review_url = base_url + "/highest-rated-coffees/page/{}/"

In [3]:
links = []
for page in range(1,123):
    listing_url = review_url.format(page)
    try:
        response = requests.get(listing_url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        for a in soup.select('a[title^="Permanent Link To"]'):
            detail_url = a["href"]
            if detail_url not in links:
                links.append(detail_url)
        time.sleep(1)
    except Exception as e:
        print(f"Error on page {page}: {e}")
        continue


In [5]:
len(links)

2217

In [ ]:
def parse_review(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")

        coffee_name = soup.find("h1").text.strip()

        score = soup.find("span", class_="review-template-rating").text.strip()

        table = soup.find("table", class_="review-template-table")
        rows = table.find_all("tr")
        data = { "coffee_name": coffee_name, "total_score": score }

        for row in rows:
            cols = row.find_all("td")
            if len(cols) == 2:
                label = cols[0].text.strip().replace(":", "").lower().replace(" ", "_")
                value = cols[1].text.strip()
                data[label] = value
        return data
    except Exception as e:
        print(f"Error parsing {url}: {e}")
        return None

In [7]:
all_reviews = []
for i, url in enumerate(links):
    review_data = parse_review(url)
    if review_data:
        all_reviews.append(review_data)
    time.sleep(1)

In [9]:
df = pd.DataFrame(all_reviews)
df.to_csv("top-rated-coffee.csv", index=False)